# Model

not related to model but good to put at the top

restart runtime after

In [0]:
#some dtype problem that came out only in 0.23 for get_dummies
!pip3 install --upgrade pandas

    100% |████████████████████████████████| 8.9MB 4.7MB/s 
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.1 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0
  [pandas]
You must restart the runtime in order to use newly installed versions.


In [0]:
import keras.backend as K

Using TensorFlow backend.


In [0]:
import tensorflow as tf

In [0]:
import keras.layers as KL
#from keras import losses
from keras.models import Model

In [0]:
def conv1d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=True,
              name=None):
    """Utility function to apply conv + BN.
    Obtained from:
    github.com/keras-team/keras-applications/blob/master/keras_applications/inception_resnet_v2.py
    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        kernel_size: kernel size as in `Conv2D`.
        strides: strides in `Conv2D`.
        padding: padding mode in `Conv2D`.
        activation: activation in `Conv2D`.
        use_bias: whether to use a bias in `Conv2D`.
        name: name of the ops; will become `name + '_ac'` for the activation
            and `name + '_bn'` for the batch norm layer.
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    x = KL.Conv1D(filters,
                  kernel_size,
                  strides=strides,
                  padding=padding,
                  use_bias=use_bias,
                  name=name)(x)

    bn_name = None if name is None else name + '_bn'
    # training set to False according to some code for Mask RCNN by matterport
    x = KL.BatchNormalization(axis=2,
                              scale=False,
                              name=bn_name)(x, training=False)
    if activation is not None:
        ac_name = None if name is None else name + '_ac'
        x = KL.Activation(activation, name=ac_name)(x)
    return x

In [0]:
# p_size usually 3 in our facenet case
def inception_tower_4_pool(x, f1x1, f3x3r, f3x3, f5x5r, f5x5, p_size, f1x1p, prefix):

    tower_1 = conv1d_bn(x, f1x1, (1,), name=prefix + '_conv_1')

    tower_2 = conv1d_bn(x, f3x3r, (1,), name=prefix + '_conv_2')
    tower_2 = conv1d_bn(tower_2, f3x3, (3, ), name=prefix + 'conv_3')

    tower_3 = conv1d_bn(x, f5x5r, (1,), name=prefix + '_conv_4')
    tower_3 = conv1d_bn(tower_3, f5x5, (5, ), name=prefix + 'conv_5')

    tower_4 = KL.MaxPooling1D((p_size,), strides=(1,), padding='same', name=prefix + '_max_pool')(x)
    tower_4 = conv1d_bn(tower_4, f1x1p, (1,), name=prefix + '_conv_6')

    return KL.concatenate([tower_1, tower_2, tower_3, tower_4], axis=2, name=prefix + '_cat')
  
def inception_tower_4_l2(x, f1x1, f3x3r, f3x3, f5x5r, f5x5, f1x1p, prefix):

    tower_1 = conv1d_bn(x, f1x1, (1,), name=prefix + '_conv_1')

    tower_2 = conv1d_bn(x, f3x3r, (1,), name=prefix + '_conv_2')
    tower_2 = conv1d_bn(tower_2, f3x3, (3, ), name=prefix + 'conv_3')

    tower_3 = conv1d_bn(x, f5x5r, (1,), name=prefix + '_conv_4')
    tower_3 = conv1d_bn(tower_3, f5x5, (5, ), name=prefix + 'conv_5')

    tower_4 = KL.Lambda(K.l2_normalize, name=prefix + '_L2')(x)
    tower_4 = conv1d_bn(tower_4, f1x1p, (1,), name=prefix + '_conv_6')

    return KL.concatenate([tower_1, tower_2, tower_3, tower_4], axis=2, name=prefix + '_cat')

# note that strides for max pool is 2 here and without dim. reduction
# and 3x3 & 5x5 also have stride 2
# and there is no conv_1 (or conv_6)
def inception_tower_3(x, f3x3r, f3x3, f5x5r, f5x5, p_size, prefix):

    tower_2 = conv1d_bn(x, f3x3r, (1,), name=prefix + '_conv_2')
    tower_2 = conv1d_bn(tower_2, f3x3, (3, ), strides=(2,), name=prefix + 'conv_3')

    tower_3 = conv1d_bn(x, f5x5r, (1,), name=prefix + '_conv_4')
    tower_3 = conv1d_bn(tower_3, f5x5, (5, ), strides=(2,), name=prefix + 'conv_5')

    tower_4 = KL.MaxPooling1D((p_size,), strides=(2,), padding='same', name=prefix + '_max_pool')(x)

    return KL.concatenate([tower_2, tower_3, tower_4], axis=2, name=prefix + '_cat')

In [0]:
def local_norm_1d(x):
  
  x = K.expand_dims(x, axis=0)
  x = tf.nn.local_response_normalization(x)
  return K.squeeze(x, axis=0)

In [0]:
model_in = KL.Input((300, 1), name='the_input')
target_in = KL.Input((20,), name='target_in')
diff_in = KL.Input((1,), name='diff_in')

In [0]:
x = KL.ZeroPadding1D(10, name='padding_1')(model_in)

x = conv1d_bn(x, 64, (7,), strides=2, name='conv_1')

x = KL.MaxPooling1D(3, strides=2, padding='same', name='max_pool_1')(x)
x = KL.Lambda(local_norm_1d, name='norm_1')(x)

x = conv1d_bn(x, 64, (1,), name='ncptn_1_conv_1')
x = conv1d_bn(x, 192, (3,), name='ncptn_1_conv_2')
x = conv1d_bn(x, 64, (1,), name='ncptn_2_conv_1')
x = conv1d_bn(x, 192, (3,), name='ncptn_2_conv_2')

x = KL.Lambda(local_norm_1d, name='norm_2')(x)
x = KL.MaxPooling1D(3, strides=2, padding='same', name='max_pool_2')(x)

x = inception_tower_4_pool(x, 64, 96, 128, 16, 32, 3, 32, prefix='ncptn_3a')
x = inception_tower_4_l2(  x, 64, 96, 128, 32, 64,    64, prefix='ncptn_3b')
x = inception_tower_3(     x,    128, 256, 32, 64, 3,     prefix='ncptn_3c')

x = inception_tower_4_l2(x, 256,  96, 192, 32, 64, 128, prefix='ncptn_4a')
x = inception_tower_4_l2(x, 224, 112, 224, 32, 64, 128, prefix='ncptn_4b')
x = inception_tower_4_l2(x, 192, 128, 256, 32, 64, 128, prefix='ncptn_4c')
x = inception_tower_4_l2(x, 160, 144, 288, 32, 64, 128, prefix='ncptn_4d')
x = inception_tower_3(   x,      160, 256, 64,128,    3,prefix='ncptn_4e')

x = inception_tower_4_l2(  x, 384, 192, 384, 48, 128,    128, prefix='ncptn_5a')
x = inception_tower_4_pool(x, 384, 192, 384, 48, 128, 3, 128, prefix='ncptn_5b')

x = KL.AveragePooling1D(10, name='avg_pool_1')(x)
x = KL.Flatten()(x)

# inception v1 does dropout, described in "Going Deeper with Conv..." is 40%,
# but tf impl. defaults to 20%
x = KL.Dropout(0.4)(x)

d1 = KL.Dense(20, name='dense_1')(x)
d2 = KL.Dense(20, name='dense_2')(x)
d3 = KL.Dense(20, name='dense_3')(x)
d4 = KL.Dense(20, name='dense_4')(x)

# d1 = KL.Dense(20, activation='softmax', name='dense_1')(x)
# d2 = KL.Dense(20, activation='softmax', name='dense_2')(x)
# d3 = KL.Dense(20, activation='softmax', name='dense_3')(x)
# d4 = KL.Dense(20, activation='softmax', name='dense_4')(x)

In [0]:
def model_loss_fn(y_true, y_pred, difficulties, diff_level):

  indices = tf.where(tf.equal(difficulties, diff_level))

  actual = tf.gather(y_true, indices)
  predict = tf.gather(y_pred, indices)

  return tf.losses.softmax_cross_entropy(actual, predict)

In [0]:
loss_1 = KL.Lambda(lambda x: model_loss_fn(*x, 1), name='loss_1')(
                   [target_in, d1, diff_in])
loss_2 = KL.Lambda(lambda x: model_loss_fn(*x, 2), name='loss_2')(
                   [target_in, d2, diff_in])
loss_3 = KL.Lambda(lambda x: model_loss_fn(*x, 3), name='loss_3')(
                   [target_in, d3, diff_in])
loss_4 = KL.Lambda(lambda x: model_loss_fn(*x, 4), name='loss_4')(
                   [target_in, d4, diff_in])

In [0]:
inputs = [model_in, target_in, diff_in]

In [0]:
outputs = [d1, d2, d3, d4, loss_1, loss_2, loss_3, loss_4]

In [0]:
model = Model(inputs, outputs)

In [0]:
model.add_loss(loss_1)
model.add_loss(loss_2)
model.add_loss(loss_3)
model.add_loss(loss_4)

## visualization

In [0]:
from keras.utils import plot_model

In [0]:
plot_model(model, to_file='model.png')

In [0]:
from google.colab import files

In [0]:
files.download('model.png')

## sanity check

loss fn 2.0 test edge case

In [0]:
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

In [0]:
def model_loss_fn(y_true, y_pred, difficulties, diff_level):

  indices = tf.where(tf.equal(difficulties, diff_level))

  actual = tf.gather(y_true, indices)
  predict = tf.gather(y_pred, indices)

  return tf.losses.softmax_cross_entropy(actual, predict)

In [0]:
import numpy as np

In [0]:
foo_true = np.array([[[0, 1, 0, 0],
                     [1, 0, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]
                    ]], dtype=np.float32)
foo_pred = np.array([[[0, 0, 1, 0],
                     [0, 0, 0, 1],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0]
                    ]], dtype=np.float32)
foo_diff = np.array([0,
                     1,
                     0,
                     1
                    ], dtype=np.float32)

In [0]:
model_loss_fn(foo_true, foo_pred, foo_diff, np.array([2], dtype=np.float32))

<tf.Tensor: id=73, shape=(), dtype=float32, numpy=0.0>

In [0]:
true = ([0]*19)
true.append(1)
pred = [1]
pred.extend([0]*19)
tf.losses.softmax_cross_entropy(np.array(true, dtype=np.float32),
                                np.array(pred, dtype=np.float32))

<tf.Tensor: id=133, shape=(), dtype=float32, numpy=3.0781546>

In [0]:
true = ([0]*19)
true.append(1)
pred = [0.05]*20
tf.losses.softmax_cross_entropy(np.array(true, dtype=np.float32),
                                np.array(pred, dtype=np.float32))

<tf.Tensor: id=199, shape=(), dtype=float32, numpy=2.9957323>

loss fn 1 test 2

In [0]:
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

In [0]:
def model_loss_fn(y_true, y_pred, difficulties, diff_level):

  indices = tf.where(tf.equal(difficulties, diff_level))
  
#   tf.print(indices)
  
  actual = tf.gather(y_true, indices)
  predict = tf.gather(y_pred, indices)
  
#   tf.print(actual)
#   tf.print(predict)
  
  return losses.categorical_crossentropy(actual, predict)

In [0]:
import numpy as np

In [0]:
tf.losses.softmax_cross_entropy(np.ones((2, 20)), np.ones((2, 20)))

<tf.Tensor: id=67, shape=(), dtype=float64, numpy=59.91464614868164>

In [0]:
foo_true = np.array([[[0, 1, 0, 0],
                     [1, 0, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]
                    ]], dtype=np.float32)
foo_pred = np.array([[[0, 0, 1, 0],
                     [0, 0, 0, 1],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0]
                    ]], dtype=np.float32)
foo_diff = np.array([0,
                     1,
                     0,
                     1
                    ], dtype=np.float32)

In [0]:
model_loss_fn(foo_true, foo_pred, foo_diff, np.array([0], dtype=np.float32))

<tf.Tensor: id=91, shape=(2, 1), dtype=float32, numpy=
array([[16.118095],
       [16.118095]], dtype=float32)>

output shape

In [0]:
model.get_layer('dense_1').output_shape

(None, 20)

In [0]:
model.get_layer('target_in').output_shape

(None, 20)

In [0]:
model.get_layer('loss_1').output_shape

()

In [0]:
import numpy as np

In [0]:
model = Model(inputs=[model_in], outputs=[d1, d2, d3, d4])

In [0]:
model.predict(np.ones((1, 300, 1)))

[array([[[0.0501996 , 0.05014166, 0.04990169, 0.04997953, 0.04987779,
          0.0500317 , 0.04982448, 0.05004711, 0.04973657, 0.04987436,
          0.05002686, 0.05002549, 0.05023536, 0.04996005, 0.04973293,
          0.05017341, 0.05003966, 0.0502243 , 0.04983046, 0.05013706]]],
       dtype=float32),
 array([[[0.04963082, 0.04970811, 0.05015945, 0.05035815, 0.04988367,
          0.04997969, 0.05012528, 0.04970011, 0.04998784, 0.05003733,
          0.05018548, 0.05027582, 0.05009138, 0.04966076, 0.05031439,
          0.04969691, 0.05009257, 0.05000971, 0.05032682, 0.04977572]]],
       dtype=float32),
 array([[[0.05024128, 0.04980202, 0.04980695, 0.05012323, 0.04954685,
          0.05000426, 0.05007407, 0.0498806 , 0.05022278, 0.05044633,
          0.04988617, 0.05005704, 0.04993817, 0.04992321, 0.0501009 ,
          0.05022063, 0.05001197, 0.05021008, 0.04999688, 0.04950662]]],
       dtype=float32),
 array([[[0.0502107 , 0.04990887, 0.0502199 , 0.04971565, 0.05008033,
          0.

start loss test

In [0]:
from keras import losses

Using TensorFlow backend.


In [0]:
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

In [0]:
def model_loss_fn(y_true, y_pred, difficulties, diff_level):
  
  indices = tf.where(difficulties == diff_level)
  
  actual = tf.gather(y_true, indices)
  predict = tf.gather(y_pred, indices)
  
  return losses.categorical_crossentropy(actual, predict)

In [0]:
import numpy as np

In [0]:
foo_true = np.array([[0, 1, 0, 0],
                     [1, 0, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]
                    ], dtype=np.float32)
foo_pred = np.array([[0, 0, 1, 0],
                     [0, 0, 0, 1],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0]
                    ], dtype=np.float32)
foo_diff = np.array([0,
                     1,
                     0,
                     1
                    ], dtype=np.float32)

In [0]:
model_loss_fn(foo_true, foo_pred, foo_diff, np.array([0], dtype=np.float32))

<tf.Tensor: id=80, shape=(2, 1), dtype=float32, numpy=
array([[16.118095],
       [16.118095]], dtype=float32)>

In [0]:
model_loss_fn(foo_true, foo_pred, foo_diff, np.array([1], dtype=np.float32))

<tf.Tensor: id=102, shape=(2, 1), dtype=float32, numpy=
array([[16.118095],
       [16.118095]], dtype=float32)>

end loss test

In [0]:
for layer in model.layers:
  print(layer.name)
  print(layer.output_shape)

input_4
(None, 300, 1)
padding_1
(None, 308, 1)
conv_1
(None, 154, 64)
conv_1_bn
(None, 154, 64)
conv_1_ac
(None, 154, 64)
max_pool_1
(None, 77, 64)
norm_1
(None, 77, 64)


In [0]:
model_in = KL.Input((2, 2), name='the_input')

In [0]:
x = KL.Lambda(local_norm_1d, name='norm_1')(model_in)

In [0]:
model = Model(inputs=model_in, outputs=x)

In [0]:
import numpy as np

In [0]:
model.predict(np.ones((2,2,2)))

array([[[0.57735026, 0.57735026],
        [0.57735026, 0.57735026]],

       [[0.57735026, 0.57735026],
        [0.57735026, 0.57735026]]], dtype=float32)

In [0]:
foo = np.ones((2,2,2))
foo[0][0][0] = 0
foo

array([[[0., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]]])

In [0]:
model.predict(foo)

array([[[0.        , 0.7071067 ],
        [0.57735026, 0.57735026]],

       [[0.57735026, 0.57735026],
        [0.57735026, 0.57735026]]], dtype=float32)

In [0]:
foo = np.ones((2,2,2))
foo[0][0][0] = 0
foo[0][1][0] = 0
foo

array([[[0., 1.],
        [0., 1.]],

       [[1., 1.],
        [1., 1.]]])

In [0]:
model.predict(foo)

array([[[0.        , 0.7071067 ],
        [0.        , 0.7071067 ]],

       [[0.57735026, 0.57735026],
        [0.57735026, 0.57735026]]], dtype=float32)

# Data

## io

In [0]:
from google.colab import files

In [0]:
foo = files.upload()
foo = None

Saving kaggle.json to kaggle.json


In [0]:
!chmod 600 kaggle.json
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json
!kaggle competitions download -c 20-newsgroups-ciphertext-challenge
!unzip test.csv.zip
!unzip train.csv.zip

  0% 0.00/556k [00:00<?, ?B/s]
100% 556k/556k [00:00<00:00, 37.7MB/s]
 31% 5.00M/16.2M [00:00<00:01, 8.18MB/s]
100% 16.2M/16.2M [00:00<00:00, 23.7MB/s]
 72% 5.00M/6.98M [00:00<00:00, 9.47MB/s]
100% 6.98M/6.98M [00:00<00:00, 12.9MB/s]
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
!ls

sample_data		   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [0]:
!rm /root/.kaggle/kaggle.json

## process

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
train = pd.read_csv('train.csv')

In [0]:
len(train.index)

39052

In [0]:
filtered = train[train['ciphertext'].str.len() >= 45]

In [0]:
len(filtered.index)

38219

In [0]:
train_train, train_val = train_test_split(filtered, test_size=0.1,
                                          random_state=42)

In [0]:
#does not actually generate things, call get_list for data (all in memory)
class DataGenerator(object):

  def __init__(self, df):
    
    self.length = len(df.index)
    print(self.length)

    self.ciphertexts = self._transform_ciphers(df['ciphertext'])
    
    self.difficulties =  df['difficulty'].astype("float32").values
    #pd.get_dummies(df['difficulty'], dtype='float32').values
    
    self.labels = pd.get_dummies(df['target'], dtype='float32').values
    
    #self.X = np.empty((1, 300, 128, 1), dtype='float32')
    #self.m_arange = np.arange(300)
  
  def _transform_ciphers(self, series):
    
    #print(series.shape)
    np_inputs = np.zeros((self.length, 300), dtype=np.byte)
    #for i, e in series.iteritems():
    i=0
    for _, e in series.iteritems():

      encoded = e.encode("ascii", errors='strict')
      np_enc = np.frombuffer(encoded, dtype=np.byte)
      how_big = np_enc.shape[0]
      np_inputs[i, :how_big] = np_enc
      
      i+=1
      
    return np_inputs

  def get_list(self):
    
    return [self.ciphertexts[:, :, np.newaxis], self.labels, self.difficulties]

In [0]:
train_gen = DataGenerator(train_train)
val_gen = DataGenerator(train_val)

34397
3822


# Training

        for name in loss_names:
            if name in self.keras_model.metrics_names:
                continue
            layer = self.keras_model.get_layer(name)
            self.keras_model.metrics_names.append(name)
            self.keras_model.metrics_tensors.append(tf.reduce_mean(
                layer.output, keep_dims=True))

In [0]:
import keras.optimizers

In [0]:
# model.compile(optimizer='rmsprop',
#               loss=None,
#               metrics=['accuracy'])

# 8 outputs
# model.compile(optimizer='rmsprop', loss=[None]*8)

# Mask RCNN
# Learning rate and momentum
# The Mask RCNN paper uses lr=0.02, but on TensorFlow it causes
# weights to explode. Likely due to differences in optimizer
# implementation.
LEARNING_RATE = 0.001
LEARNING_MOMENTUM = 0.9
GRADIENT_CLIP_NORM = 5.0

optimizer = keras.optimizers.SGD(
            lr=LEARNING_RATE, momentum=LEARNING_MOMENTUM,
            clipnorm=GRADIENT_CLIP_NORM)

model.compile(optimizer=optimizer, loss=[None]*8)

In [0]:
print(model.metrics_names)
print(model.metrics_tensors)

['loss']

In [0]:
layer = model.get_layer("loss_1")
model.metrics_names.append("loss_1")
model.metrics_tensors.append(tf.reduce_mean(layer.output, keepdims=True))

layer = model.get_layer("loss_2")
model.metrics_names.append("loss_2")
model.metrics_tensors.append(tf.reduce_mean(layer.output, keepdims=True))

layer = model.get_layer("loss_3")
model.metrics_names.append("loss_3")
model.metrics_tensors.append(tf.reduce_mean(layer.output, keepdims=True))

layer = model.get_layer("loss_4")
model.metrics_names.append("loss_4")
model.metrics_tensors.append(tf.reduce_mean(layer.output, keepdims=True))

layer = None

In [0]:
print(model.metrics_names)
print(model.metrics_tensors)

['loss', 'loss_1', 'loss_2', 'loss_3', 'loss_4']
[<tf.Tensor 'Mean:0' shape=() dtype=float32>, <tf.Tensor 'Mean_1:0' shape=() dtype=float32>, <tf.Tensor 'Mean_2:0' shape=() dtype=float32>, <tf.Tensor 'Mean_3:0' shape=() dtype=float32>]


In [0]:
import keras.callbacks as KC

In [0]:
!mkdir ./test

In [0]:
checkpt = KC.ModelCheckpoint('./test/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1)

In [0]:
!mkdir ./logs

In [0]:
tb_call = KC.TensorBoard(log_dir="./logs",
                         histogram_freq=0, write_graph=True,
                         write_images=False)

In [0]:
i = 0

In [0]:
# hist = model.fit(x=train_gen.get_list(), y=None, batch_size=32, epochs=i+49,
#                  verbose=1, callbacks=[checkpt],
#                  validation_data=(val_gen.get_list(), None),
#                  shuffle=True, initial_epoch=i)
# i += 49
print('foo')

In [0]:
hist = model.fit(x=train_gen.get_list(), y=None, batch_size=8, epochs=i+10,
                 verbose=1, callbacks=[checkpt, tb_call],
                 validation_data=(val_gen.get_list(), None),
                 shuffle=True, initial_epoch=i)
i += 10

Train on 34397 samples, validate on 3822 samples
Epoch 13/22
34397/34397 [==============================] - 325s 9ms/step - loss: 10.6616 - loss_1: 2.6442 - loss_2: 2.6868 - loss_3: 2.6441 - loss_4: 2.6866 - val_loss: 10.6900 - val_loss_1: 2.6695 - val_loss_2: 2.6402 - val_loss_3: 2.6507 - val_loss_4: 2.7297
Epoch 14/22
34397/34397 [==============================] - 316s 9ms/step - loss: 10.6650 - loss_1: 2.6570 - loss_2: 2.6973 - loss_3: 2.6325 - loss_4: 2.6783 - val_loss: 10.6886 - val_loss_1: 2.6694 - val_loss_2: 2.6395 - val_loss_3: 2.6504 - val_loss_4: 2.7293
Epoch 15/22
34397/34397 [==============================] - 315s 9ms/step - loss: 10.6900 - loss_1: 2.6470 - loss_2: 2.6943 - loss_3: 2.6557 - loss_4: 2.6930 - val_loss: 10.6890 - val_loss_1: 2.6693 - val_loss_2: 2.6392 - val_loss_3: 2.6507 - val_loss_4: 2.7298
Epoch 16/22
34397/34397 [==============================] - 316s 9ms/step - loss: 10.6304 - loss_1: 2.6428 - loss_2: 2.6859 - loss_3: 2.6349 - loss_4: 2.6668 - val_loss:

In [0]:
type(hist)

keras.callbacks.History

In [0]:
hist

In [0]:
!ls test

weights.01-10.78.hdf5  weights.04-10.73.hdf5  weights.07-10.73.hdf5
weights.02-10.74.hdf5  weights.05-10.74.hdf5  weights.08-10.73.hdf5
weights.03-10.73.hdf5  weights.06-10.73.hdf5  weights.09-10.73.hdf5


In [0]:
!mv test test1

In [0]:
!ls logs

events.out.tfevents.1547934128.4d0da3a2cfc4


In [0]:
!ls -l test/weights.60*

-rw-r--r-- 1 root root 18688520 Jan 19 05:58 test/weights.60-11.92.hdf5


In [0]:
from google.colab import files

In [0]:
files.download('test/weights.22-10.69.hdf5')

In [0]:
!mv test/weights.09-10.73.hdf5 .

In [0]:
!ls test

weights.01-11.95.hdf5  weights.09-10.68.hdf5  weights.17-10.69.hdf5
weights.02-10.70.hdf5  weights.10-10.68.hdf5  weights.18-10.68.hdf5
weights.03-10.69.hdf5  weights.11-10.68.hdf5  weights.19-10.68.hdf5
weights.04-10.69.hdf5  weights.12-10.69.hdf5  weights.20-10.68.hdf5
weights.05-10.69.hdf5  weights.13-10.69.hdf5  weights.21-10.68.hdf5
weights.06-10.69.hdf5  weights.14-10.69.hdf5  weights.22-10.69.hdf5
weights.07-10.68.hdf5  weights.15-10.69.hdf5
weights.08-10.69.hdf5  weights.16-10.69.hdf5


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-01-20 02:25:17--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.199.255.1, 34.238.3.58, 34.204.22.7, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.199.255.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  3.46MB/s    in 1.5s    

2019-01-20 02:25:19 (3.46 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!curl -s http://localhost:4040/api/tunnels

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!pgrep tensorboard

2296


In [0]:
!pgrep ngrok

In [0]:
!kill -SIGINT `pgrep ngrok`

In [0]:
!kill -SIGTERM `pgrep tensorboard`

In [0]:
!pgrep tensorboard

In [0]:
!pgrep ngrok